In [18]:
import pandas as pd
import tkinter as tk

from datetime import datetime
from smartapi import SmartConnect
from symbol_selector import UserInterface


In [19]:
uname = ""
pwd = ""
api_key = "CGz5s9Tv"
api_key_feed = "bgBHbiTo"
secret = "1c873097-b4f6-4b50-ae5f-5b70341a0347"
tokens_url = "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json"

df_master = pd.read_json(tokens_url)
s_map = pd.Series(df_master.name.values,index=df_master.symbol).to_dict()
s_concat_list = ["{}  ||  {}".format(key, value) for key, value in s_map.items()]


In [20]:
obj = SmartConnect(api_key=api_key)
data = obj.generateSession(uname, pwd)
refresh_token = data['data']['refreshToken']
feedtoken = obj.getfeedToken()
user_profile = obj.getProfile(refresh_token)

In [21]:
def get_historic_data(exchange, symbol, token, interval, fdate, todate):
    try:
        params={
            "exchange": exchange,
            "tradingsymbol": symbol,
            "symboltoken": token,
            "interval": interval,
            "fromdate": fdate,
            "todate": todate
        }
        candle_data = obj.getCandleData(params)['data']
        df_candle = pd.DataFrame(candle_data)
        
        df_candle = df_candle.rename(
                    columns = {0: "Datetime", 1: "Open", 2: "High", 3: "Low", 4: "Close", 5: "Volume"}
                   )
        df_candle['Datetime'] = pd.to_datetime(df_candle['Datetime'])
        df_candle = df_candle.set_index('Datetime')
        return df_candle
    except Exception as e:
        print("Something went wrong - {}".format(e))
        

In [22]:
root = tk.Tk()
root.title('Select a Symbol')
root.geometry("500x400")
ui_obj = UserInterface(root, s_concat_list)
root.mainloop()

selected_symbol = ui_obj.get_symbol_value().split('  ||  ')[0]
ser = df_master.loc[df_master['symbol'] == selected_symbol]
selected_token = str(ser['token'].to_list()[0])

exchange = 'NSE'
timeframe = "ONE_DAY"
from_date = "2021-09-01 00:00"
to_date = "2021-09-08 23:59"
daily_data = get_historic_data(exchange, selected_symbol, selected_token, timeframe, from_date, to_date)
result_df = pd.DataFrame(daily_data)
# my_df.to_csv('{}.csv'.format('sample_oneday', mode='w', index=True))

curr_time = datetime.now()
ctime = curr_time.strftime("%Y-%m-%d_%H-%M-%S")
result_df.to_csv("{}_{}.csv".format(selected_symbol, ctime))
print("csv file has been created for {} at {}".format(selected_symbol, ctime))

        

csv file has been created for SBIN-EQ at 2021-09-13_18-44-09
